In [1]:
from Bio import Phylo
from io import StringIO
import numpy as np

In [2]:
from simulate_tree import simulate_tree

In [5]:
# sample_size_1 = 2
# sample_size_2 = 0
# transition_rate = 0.1
# relative_seedbank_size = 1
# theta = 1

In [4]:
# # cs = [0.1, 0.3]
# # Ks = [0.3, 0.6, 1, 3]
# # thetas = [0.5, 1, 2]

# sample_size_1 = 5
# sample_size_2 = 0
# cs = [0.1]
# Ks = [1]
# thetas = [1]

In [ ]:
it = 1000000
res = {}
for c in cs:
    for K in Ks:
        for theta in thetas:
            sum_tmrca = 0
            for i in range(it):
                out = simulate_tree(sample_size_1, sample_size_2, c, K, theta)
                root, island, time_vector, anc, dec_1, dec_2, log = out
                sum_tmrca += out[2][-1]
            tmrca = sum_tmrca/it
            print(tmrca)
            res[(c, K, theta)] = tmrca

In [ ]:
it = 1000000
c=1.0
K=1.0
theta=1.0
sample_size_1=5

values=[]
sum_tmrca = 0
for i in range(it):
    out = simulate_tree(sample_size_1, sample_size_2, c, K, theta)
    root, island, time_vector, anc, dec_1, dec_2, log = out
    values.append(out[2][-1])
    sum_tmrca += out[2][-1]

var = np.var(values)
tmrca = sum_tmrca/it
print(var, tmrca)

In [3]:
# Expected TMRCA
# Want to solve:
# lambda(n, m) t(n, m) = 1 + (alpha_1(n, m) * t(n-1, m)
#                           + alpha_2(n, m) * t(n, m-1))
#                           + beta(n, m) * t(n-1, m+1)
#                           + gamma(n, m) * t(n+1, m-1)
#Initial conditions: t(0, 1) = t(1, 0) = 0
def lamb(n, m):
    return (n) * (n-1)/2 + kappa * (m) * (m-1)/2 + c * n + c*K*m
def alpha_1(n, m):
    return (n) * (n-1)/2
def alpha_2(n, m):
    return kappa * (m) * (m-1)/2
def beta (n, m):
    return c*n
def gamma (n, m):
    return c*K*m
def calc_t(l, alpha_1, alpha_2, beta, gamma, lamb):
    if l == 1:
        return [0, 0]
    prev_t = calc_t(l-1, alpha_1, alpha_2, beta, gamma, lamb)
    A = np.zeros((l+1, l+1))
    for i in range(l+1):
        A[i][i] = lamb(i, l-i)
        if i > 0:
            A[i][i-1] = -beta(i, l-i)
        if i < l:
            A[i][i+1] = -gamma(i, l-i)
    b = np.zeros(l+1)
    for i in range(l+1):
        b[i] = 1
        if i > 0:
            b[i] += alpha_1 (i, l-i) * prev_t[i-1]
        if i < l:
            b[i] += alpha_2 (i, l-i) * prev_t[i]
    return np.linalg.solve(A, b)

def extract(n, alpha_1, alpha_2, beta, gamma, lamb):
    return calc_t(n, alpha_1, alpha_2, beta, gamma, lamb)[n]

kappa=1
# for K in [0.01, 1, 100]:
for K in [0.1, 1, 10]:
    for c in [0.1, 0.5, 1, 5, 10]:
        for n in [10]:
            print(f'{c} {K} {n} {extract(n, alpha_1, alpha_2, beta, gamma, lamb)}')

0.1 0.1 10 5.890981405752168
0.5 0.1 10 3.98431956453368
1 0.1 10 3.2045805491920434
5 0.1 10 2.3731106810022964
10 0.1 10 2.2637220612059834
0.1 1 10 4.368700347146745
0.5 1 10 3.9572395222642305
1 1 10 3.8030715742433343
5 1 10 3.6414785599655337
10 1 10 3.620455304641656
0.1 10 10 2.2365912599866746
0.5 10 10 2.182505216831504
1 10 10 2.1705292758547223
5 10 10 2.159456684372717
10 10 10 2.1579605584190964


In [7]:
c = 1
K = 10
n = 25
extract(n, alpha_1, alpha_2, beta, gamma, lamb)

2.354165029155353

In [3]:
# Variance TMRCA
def L(n, m):
    return n * (n-1)/2 + c * n + c * K * m

def A(n, m):
    return n * (n-1)/2 / L(n, m)

def B(n, m):
    return c * n / L(n, m)

def G(n, m):
    return c * K * m / L(n, m)

def FindTV(l):
    if l == 1:
        return [0, 0], [0, 0]

    Prev_T, Prev_V = FindTV(l-1)

    M = np.zeros((l + 1, l + 1))
    for i in range(l + 1):
        M[i][i] = 1
        if i > 0:
            M[i][i - 1] = -B(i, l - i)
        if i < l:
            M[i][i + 1] = -G(i, l - i)

    # Step 4: Define the vector b
    b = np.zeros(l + 1)
    for i in range(l+1):
        b[i] = 1/L(i, l-i)
        if i > 0:
            b[i] += A(i, l-i) * Prev_T[i-1]

    T = np.linalg.solve(M, b)
    # print(f"{T}")
         
    # Define the vector c
    c_vec = np.zeros(l + 1)
    for i in range(l + 1): 
        if i == 0:
            c_vec[i] = 1/L(0, l)**2 \
                + G(0, l) * T[1]**2 \
                - (G(0, l) * T[1])**2
        elif i == l:
            c_vec[i] = 1/L(l, 0)**2 \
                + A(l, 0) * Prev_V[l - 1] \
                + A(l, 0) * Prev_T[l - 1] ** 2 \
                + B(l, 0) * T[l - 1] ** 2 \
                - (A(l, 0) * Prev_T[l - 1] \
                   + B(l, 0) * T[l - 1]) ** 2
        else:
            c_vec[i] = 1/L(i, l-i)**2 \
                + A(i, l - i) * Prev_V[i - 1] \
                + A(i, l - i) * Prev_T[i - 1] ** 2 \
                + B(i, l - i) * T[i - 1] ** 2 \
                + G(i, l - i) * T[i + 1] ** 2 \
                - (A(i, l - i) * Prev_T[i - 1] \
                   + B(i, l - i) * T[i - 1] \
                   + G(i, l - i) * T[i + 1]) ** 2

    V = np.linalg.solve(M, c_vec)

    return T, V

def extract_var(l):
    T, V = FindTV(l)
    return T[l], V[l]


for K in [1, 10]:
# for K in [1]:
    for c in [1]:
    # for c in [0.5]:
        for l in [25]:
            T, V = extract_var(l)
            print(f"{V}")

35.51040151411206
1.7664602129589047


In [7]:
# ACTIVE LENGTH
def L(n, m):
    return n * (n-1)/2 + c * n + c * K * m

def A(n, m):
    return n * (n-1)/2 / L(n, m)

def B(n, m):
    return c * n / L(n, m)

def G(n, m):
    return c * K * m / L(n, m)

def FindTV(l):
    if l == 1:
        return [0, 0], [0, 0]

    Prev_T, Prev_V = FindTV(l-1)

    M = np.zeros((l + 1, l + 1))
    for i in range(l + 1):
        M[i][i] = 1
        if i > 0:
            M[i][i - 1] = -B(i, l - i)
        if i < l:
            M[i][i + 1] = -G(i, l - i)

    # Step 4: Define the vector b
    b = np.zeros(l + 1)
    for i in range(l+1):
        b[i] = i * 1/L(i, l-i)
        if i > 0:
            b[i] += A(i, l-i) * Prev_T[i-1]

    T = np.linalg.solve(M, b)
    # print(f"{T}")
         
    # Define the vector c
    c_vec = np.zeros(l + 1)
    for i in range(l + 1): 
        if i == 0:
            c_vec[i] = i**2/L(0, l)**2 \
                + G(0, l) * T[1]**2 \
                - (G(0, l) * T[1])**2
        elif i == l:
            c_vec[i] = i**2/L(l, 0)**2 \
                + A(l, 0) * Prev_V[l - 1] \
                + A(l, 0) * Prev_T[l - 1] ** 2 \
                + B(l, 0) * T[l - 1] ** 2 \
                - (A(l, 0) * Prev_T[l - 1] \
                   + B(l, 0) * T[l - 1]) ** 2
        else:
            c_vec[i] = i**2/L(i, l-i)**2 \
                + A(i, l - i) * Prev_V[i - 1] \
                + A(i, l - i) * Prev_T[i - 1] ** 2 \
                + B(i, l - i) * T[i - 1] ** 2 \
                + G(i, l - i) * T[i + 1] ** 2 \
                - (A(i, l - i) * Prev_T[i - 1] \
                   + B(i, l - i) * T[i - 1] \
                   + G(i, l - i) * T[i + 1]) ** 2

    V = np.linalg.solve(M, c_vec)

    return T, V

def extract_var(l):
    T, V = FindTV(l)
    return T[l], V[l]


for K in [0.1, 1, 10]:
# for K in [1]:
    for c in [0.1, 0.5, 1, 5, 10]:
    # for c in [0.5]:
        for l in [2, 10, 100]:
            T, V = extract_var(l)
            print("{:0.4f}".format(T), "{:0.4f}".format(V))

22.0000 4484.0000
61.1545 11077.1340
105.3024 16489.5898
22.0000 1284.0000
59.2686 2424.1502
92.9337 2746.2820
22.0000 884.0000
58.8056 1500.0975
90.3704 1631.1341
22.0000 564.0000
60.0805 879.7547
95.5435 942.1819
22.0000 524.0000
60.8830 811.8233
99.9194 866.7038
4.0000 56.0000
11.2248 128.7470
19.9783 187.0649
4.0000 24.0000
11.1399 42.7789
19.3744 49.8455
4.0000 20.0000
11.1469 33.3213
19.3997 37.4716
4.0000 16.8000
11.2386 26.3515
19.9148 28.5460
4.0000 16.4000
11.2708 25.4995
20.1565 27.3972
2.2000 5.2400
6.2207 8.5375
11.3636 9.7483
2.2000 4.9200
6.2217 7.6674
11.3695 8.2775
2.2000 4.8800
6.2224 7.5602
11.3750 8.0978
2.2000 4.8480
6.2234 7.4741
11.3849 7.9510
2.2000 4.8440
6.2236 7.4633
11.3872 7.9320


In [11]:
# DORMANT LENGTH
def L(n, m):
    return n * (n-1)/2 + c * n + c * K * m

def A(n, m):
    return n * (n-1)/2 / L(n, m)

def B(n, m):
    return c * n / L(n, m)

def G(n, m):
    return c * K * m / L(n, m)

def FindTV(l):
    if l == 1:
        return [0, 0], [0, 0]

    Prev_T, Prev_V = FindTV(l-1)

    M = np.zeros((l + 1, l + 1))
    for i in range(l + 1):
        M[i][i] = 1
        if i > 0:
            M[i][i - 1] = -B(i, l - i)
        if i < l:
            M[i][i + 1] = -G(i, l - i)

    # Step 4: Define the vector b
    b = np.zeros(l + 1)
    for i in range(l+1):
        b[i] = (l-i)/L(i, l-i)
        if i > 0:
            b[i] += A(i, l-i) * Prev_T[i-1]

    T = np.linalg.solve(M, b)
    # print(f"{T}")
         
    # Define the vector c
    c_vec = np.zeros(l + 1)
    for i in range(l + 1): 
        if i == 0:
            c_vec[i] = (l-i)**2/L(0, l)**2 \
                + G(0, l) * T[1]**2 \
                - (G(0, l) * T[1])**2
        elif i == l:
            c_vec[i] = (l-i)**2/L(l, 0)**2 \
                + A(l, 0) * Prev_V[l - 1] \
                + A(l, 0) * Prev_T[l - 1] ** 2 \
                + B(l, 0) * T[l - 1] ** 2 \
                - (A(l, 0) * Prev_T[l - 1] \
                   + B(l, 0) * T[l - 1]) ** 2
        else:
            c_vec[i] = (l-i)**2/L(i, l-i)**2 \
                + A(i, l - i) * Prev_V[i - 1] \
                + A(i, l - i) * Prev_T[i - 1] ** 2 \
                + B(i, l - i) * T[i - 1] ** 2 \
                + G(i, l - i) * T[i + 1] ** 2 \
                - (A(i, l - i) * Prev_T[i - 1] \
                   + B(i, l - i) * T[i - 1] \
                   + G(i, l - i) * T[i + 1]) ** 2

    V = np.linalg.solve(M, c_vec)

    return T, V

def extract_var(l):
    T, V = FindTV(l)
    return T[l], V[l]


for K in [0.1, 1, 10]:
# for K in [1]:
    for c in [0.1, 0.5, 1, 5, 10]:
    # for c in [0.5]:
        for l in [100]:
            T, V = extract_var(l)
            print("{:0.4f}".format(T), "{:0.4f}".format(V))

1053.0242 2225423.2055
929.3371 374415.5576
903.7037 212138.7046
955.4345 104896.9546
999.1936 92294.9772
19.9783 957.2505
19.3744 194.5805
19.3997 110.0834
19.9148 43.8615
20.1565 35.2425
1.1364 2.7647
1.1369 0.6170
1.1375 0.3486
1.1385 0.1332
1.1387 0.1062
